In [5]:
import sys
# sys.path.append('/Users/geohacker/utils/env/kelpwatch/lib/python3.9/site-packages')
import geopandas as gpd
from lonboard import Map, PolygonLayer
from geoarrow.rust.core import read_parquet
from matplotlib.colors import LogNorm
from palettable.colorbrewer.sequential import Oranges_9, Reds_9
from lonboard.colormap import apply_continuous_cmap
from lonboard.basemap import CartoBasemap
import ipywidgets as widgets
import pandas as pd
from IPython.display import display
from lonboard.basemap import CartoBasemap
from geoarrow.rust.core import from_geopandas
import requests

In [6]:
def generate_date_strings(start_year, end_year):
    date_range = pd.date_range(start=f"{start_year}-01-01", end=f"{end_year}-04-01", freq='MS')
    date_strings = [date.strftime("%Y_%m") for date in date_range if date.month in [1, 2, 3, 4]]
    return date_strings

dates = generate_date_strings(1984, 2023)

In [13]:
# Load datasets

base_url = "https://kelpwatch-data-staging.s3.us-west-1.amazonaws.com/debug"
pixels_url = f"{base_url}/california.parquet"
hex_url = f""

with requests.get(pixels_url, stream=True) as r:
    r.raise_for_status()
    with open('california.parquet', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)

pixels = read_parquet("california.parquet")
# pixels = read_parquet(f"{data_url}/california.parquet")
pixels_gdf = pixels.to_geopandas()

hex = {}
for res in ['00', '03', '05', '06', '07', '09']:
    # hex_path = f"{base_url}/california_h3_{res}.geojson"
    # print(f"Fetching {res}")
    hex_url = f"{base_url}/california_h3_{res}.parquet"
    # hex[res] = gpd.read_file(hex_path, driver='GeoJSONSeq')
    with requests.get(hex_url, stream=True) as r:
        r.raise_for_status()
        with open(f"california_{res}.parquet", 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)
    hex[res] = read_parquet(f"california_{res}.parquet").to_geopandas()
    


Fetching 00
Fetching 03
Fetching 05
Fetching 06
Fetching 07
Fetching 09


In [14]:
m = Map(layers=[], basemap_style=CartoBasemap.Positron)
m._height = 1500

In [33]:
year_dropdown = widgets.Dropdown(
    options=dates,
    value=dates[0],
    description='YYYY_QQ:',
    disabled=False,
)

hex_dropdown = widgets.Dropdown(
    options=['00', '03', '05', '06', '07', '09'],
    value='03',
    description='HEX',
    disabled=False
)

year_qq = dates[0]
res = '09'
pixels_layer = None

def update_map(year_qq = '1984_01', res = '03'):

    normalizer = LogNorm()

    # setup pixels data and layer
    this_pixels_gdf = pixels_gdf[pixels_gdf[year_qq] >= 0]
    min = this_pixels_gdf[year_qq].min()
    max = this_pixels_gdf[year_qq].max()
    normalized_values = normalizer(this_pixels_gdf[year_qq])
    colors = apply_continuous_cmap(normalized_values, Oranges_9)
    pixels_arrow = from_geopandas(this_pixels_gdf.loc[:, [year_qq, 'geometry']])
    pixels_layer = PolygonLayer(
        table=pixels_arrow,
        get_fill_color=colors
    )

    # setup hex data and layer
    this_hex_gdf = hex[res][hex[res][year_qq] >= 0]
    hex_normalized_values = normalizer(this_hex_gdf[year_qq])
    hex_colors = apply_continuous_cmap(this_hex_gdf[year_qq], Reds_9)
    hex_layer = PolygonLayer.from_geopandas(
        this_hex_gdf.loc[:, [year_qq, 'geometry']],
        get_fill_color=hex_colors,
        opacity=0.2
    )


    m.layers = [hex_layer, pixels_layer]
    
def on_year_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"Selected date: {change['new']}")
        year_qq = change['new']
        update_map(year_qq, hex_dropdown.value)


hex_gdf = None
def on_hex_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(f"Selected hex: {change['new']}")
        res = change['new']
        update_map(year_dropdown.value, res)
    
hex_dropdown.observe(on_hex_change)
year_dropdown.observe(on_year_change)

display(year_dropdown)
display(hex_dropdown)

update_map()

Dropdown(description='YYYY_QQ:', options=('1984_01', '1984_02', '1984_03', '1984_04', '1985_01', '1985_02', '1…

Dropdown(description='HEX', index=1, options=('00', '03', '05', '06', '07', '09'), value='03')

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(
/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


In [28]:
m

Map(basemap_style=<CartoBasemap.Positron: 'https://basemaps.cartocdn.com/gl/positron-gl-style/style.json'>, vi…